# Qualitative Evaluation
## Ausgangslage
* Es sollen Kantonsratsprotokolle mit grosso-modo zeitgenössischer Sprache (ab 1950) klassifiziert werden
* Bevor ein eigenes NER-Modell trainiert oder ein bestehendes verfeinert werden soll, sind bestehende Modelle out-of-the-box zu evaluieren
* In Frage kommen grundsätzlich fertige Frameworks (vorzuziehen), gefolgt von HF-Modellen mit vortrainiertem NER-Head und HF-Modelle ohne NER-Head 
* Im Folgenden wird ein informeller Test an drei Dokumenten ausgeführt, d.h. noch ohne Goldstandardannotation.
    * Die NEs werden innerhalb des Textes visualisiert und bezüglich Label und Type ausgezählt (dieses Notebook)
    * Laufzeiten pro Dokument werden gemessen (allesamt auf lokaler Apple Mx MPS-GPU)
    * Die Resultate werden zwischen den Modellen gedifft (Schlüsse dieses Notebook unten)
* Zunächst werden Dokumente aus einem Jahr (1995) evaluiert. Die gesampelten Dokumente weisen unterschiedliche Längen auf, was die Skalierungsfähigkeit der Modelle testet.
        
## Hypothesen
Es werden folgende vortrainierten NER-Modelle evaluiert, die aufgrund einer Bereitstellung als Framework oder durch vortrainierte NER-Heads direkt anwendbar sind. Dies ist eine pragmatische Entscheidung.

* spaCy: Anwendungstaugliches NLP-Framework mit sehr guter Dokumentation, Support, usw.
    * Textchunking integriert
    * de_core_news_md, de_core_news_lg: grösste spaCy-Modelle (de_dep_news_trf hat out-of-the-box keine NER)
* HF ZurichNLP/swissbert-ner: NER, das auf Schweizer Medientexten basiert und cross-linguale Bezüge ausnützt: Annahme, dass dies zumindest teilweise auf Schweiz-spezifische Dokumente anderer Domänen übertragbar ist, bzw. dass der Schweiz-spezifische Vorteil bis zu einem gewissen Grad erhalten bleibt
* HF SpanMarker: 2022 SOTA, bietet diverse dt. Modelle
    * stefan-it/span-marker-gelectra-large-germeval14
    * gwlms/span-marker-teams-germeval14
    * gwlms/span-marker-bert-germeval14
    * gwlms/span-marker-token-dropping-bert-germeval14
* flairNLP: Alternative zu spaCy, NLP-Framework mit div. dt. NER-Modellen
        
## Vorgehensweise
* Per Sampling via Train-Test-Split (fixer Seed) selektierte Dokumente:
  * KRP/KRP_1898-1995/MM_24_142/MM_24_142_KRP_1995_223_0003.xml
  * KRP/KRP_1898-1995/MM_24_142/MM_24_142_KRP_1995_227_0003.xml
  * KRP/KRP_1898-1995/MM_24_142/MM_24_142_KRP_1995_231_0009_MM_24_142_KRP_1995_231_0010.xml          

In [ ]:
import json
import time
import xml.etree.ElementTree as ET
from collections import Counter, defaultdict
from datetime import datetime

import flair
import spacy
import torch
from bs4 import BeautifulSoup
from flair.data import Sentence
from flair.models import SequenceTagger
from IPython.display import Markdown, display
from langchain.text_splitter import SpacyTextSplitter
from spacy import displacy
from spacy.tokens import Doc
from staatsarchiv_utils import generic_text_cleaning
from tqdm.notebook import tqdm
from transformers import pipeline

spacy.require_gpu()
mps_device = torch.device("mps")
flair.device = "mps:0"


def get_ent_json(doc):
    """Produce string with pseudo-tags for NEs that can be easily diffed."""
    doc_json = doc.to_json()
    for ent in doc_json["ents"][::-1]:
        doc_json["text"] = (
            doc_json["text"][: ent["start"]]
            + r"{{{}|{}}}".format(
                doc_json["text"][ent["start"] : ent["end"]], ent["label"]
            )
            + doc_json["text"][ent["end"] :]
        )
    return doc_json["text"]

# Installation spaCy-Modelle

In [ ]:
!python -m spacy download de_core_news_md
!python -m spacy download de_core_news_lg
!python -m spacy download de_dep_news_trf

# Modellinstanziierung

## Modelle mit spaCy-Integration

In [ ]:
sm_gelectra = spacy.load("de_dep_news_trf", exclude=["ner"])
sm_gelectra.add_pipe(
    "span_marker",
    config={
        "model": "stefan-it/span-marker-gelectra-large-germeval14",
        "device": "mps",
    },
)
print(sm_gelectra.pipe_names)

sm_teams = spacy.load("de_dep_news_trf", exclude=["ner"])
sm_teams.add_pipe(
    "span_marker",
    config={"model": "gwlms/span-marker-teams-germeval14", "device": "mps"},
)

sm_bert = spacy.load("de_dep_news_trf", exclude=["ner"])
sm_bert.add_pipe(
    "span_marker",
    config={"model": "gwlms/span-marker-bert-germeval14", "device": "mps"},
)

sm_td_bert = spacy.load("de_dep_news_trf", exclude=["ner"])
sm_td_bert.add_pipe(
    "span_marker",
    config={
        "model": "gwlms/span-marker-token-dropping-bert-germeval14",
        "device": "mps",
    },
)

spacy_integrated_models = {
    "de_core_news_md": spacy.load("de_core_news_md"),
    "de_core_news_lg": spacy.load("de_core_news_lg"),
    "sm_gelectra": sm_gelectra,
    "sm_teams": sm_teams,
    "sm_bert": sm_bert,
    "sm_td_bert": sm_td_bert,
}

## Modelle ohne spaCy-Integration

In [4]:
# SwissBERT via transformers, as spaCy currently doesn't support X-MOD-based transformer heads
swissbert = pipeline(
    model="ZurichNLP/swissbert-ner", aggregation_strategy="simple", device="mps"
)

swissbert.model.set_default_language("de_CH")

fl_german_large = SequenceTagger.load("flair/ner-german-large")

2024-04-26 20:41:54,270 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>


# Plaintext aus XMLs extrahieren

In [5]:
texts = []

for doc in [
    "samples/MM_24_142_KRP_1995_223_0003.xml",
    "samples/MM_24_142_KRP_1995_227_0003.xml",
    "samples/MM_24_142_KRP_1995_231_0009_MM_24_142_KRP_1995_231_0010.xml",
]:
    tree = ET.parse(doc)
    root = tree.getroot()
    body = ET.tostring(
        root.findall(
            "./xmlns:text/xmlns:body",
            namespaces={"xmlns": "http://www.tei-c.org/ns/1.0"},
        )[0]
    )
    soup = BeautifulSoup(body)
    text = soup.find_all(string=True)
    text = " ".join([p.text for p in text])
    text = generic_text_cleaning(text)
    texts.append(text)

In [6]:
for text in texts:
    print(len(text.split()))

2079
1757
10508


In [7]:
result_json_dict = defaultdict(list)
nlp_blank = spacy.blank("de")

# Anwendung spaCy-integrierter Modelle

In [8]:
docs = defaultdict(list)

# Process
for idx, text in tqdm(enumerate(texts)):
    predictions = []
    print("Document #{}".format(idx))
    for model_name, nlp in spacy_integrated_models.items():
        print(model_name)
        start = time.time()
        doc = nlp(text)
        doc.ents = [ent for ent in doc.ents if ent.label_ in {"PER", "LOC", "ORG"}]
        docs[model_name].append(doc)
        stop = time.time()
        duration = stop - start
        print("Time: {:.2f} sec".format(duration))
    print()

for model_name in docs:
    result_json_dict[model_name] = [get_ent_json(doc) for doc in docs[model_name]]

0it [00:00, ?it/s]

Document #0
de_core_news_md
Time: 0.45 sec
de_core_news_lg
Time: 0.43 sec
sm_gelectra
Time: 9.40 sec
sm_teams
Time: 4.36 sec
sm_bert
Time: 3.94 sec
sm_td_bert
Time: 4.09 sec

Document #1
de_core_news_md
Time: 0.37 sec
de_core_news_lg
Time: 0.37 sec
sm_gelectra
Time: 7.44 sec
sm_teams
Time: 3.37 sec
sm_bert
Time: 3.53 sec
sm_td_bert
Time: 3.39 sec

Document #2
de_core_news_md
Time: 2.31 sec
de_core_news_lg
Time: 2.30 sec
sm_gelectra
Time: 44.80 sec
sm_teams
Time: 20.12 sec
sm_bert
Time: 19.15 sec
sm_td_bert
Time: 18.78 sec



In [10]:
for model_name in docs:
    display(Markdown("# {}".format(model_name)))
    for idx, doc in enumerate(docs[model_name]):
        display(Markdown("## Doc #{}".format(idx)))
        print(
            sorted(
                Counter((ent.label_ for ent in doc.ents)).items(),
                key=lambda x: x[0],
                reverse=True,
            )
        )
        print(
            ", ".join(
                list(
                    map(
                        lambda x: "{}/{}".format(x[0], x[1]),
                        sorted(
                            Counter((ent.text for ent in doc.ents)).items(),
                            key=lambda x: x[1],
                            reverse=True,
                        ),
                    )
                )
            )
        )
        displacy.render(doc, style="ent", jupyter=True)

# de_core_news_md

## Doc #0

[('PER', 26), ('ORG', 24), ('LOC', 28)]
Einzelinitiative/9, Zürich/3, SP/3, EVP/3, SVP/3, Veröffentlichung/2, Dr. Hans-Jakob Mosimann/2, Winterthur/2, Franken/2, Blum/2, Puure-Zmorge/2, FDP/2, Mosimann/2, Troesch/2, Kantonalpartei/2, Fehr/2, anerkanntermassen/1, Woher/1, Raphael Huber/1, Wes Brot/1, Kantonalparteien/1, Martin Forster/1, Landesrings/1, Franziska Troesch-Schnyder/1, Zollikon/1, Staat/1, Peter Reinhard/1, Kloten/1, Esther Holm/1, Grüne/1, Horgen/1, Nachher/1, Herrn Reinhard/1, Kontenpläne/1, Hans Fehr/1, Eglisau/1, Kantonalpartei Komitees/1, Private/1, Lucius Dürr/1, CVP/1, Spass/1, Hartmuth Attenhofer/1, Sozialdemokraten/1, Hotel International/1, Zürich-Oerlikon/1, Hotels International/1, Herr Dürr/1, Gottlob/1, Blocher/1, Frey/1, Ratsmitgliedern/1


## Doc #1

[('PER', 20), ('ORG', 13), ('LOC', 52)]
Einzelinitiative/14, Kanton Zürich/7, SVP/4, Luzern/3, Zürich/3, Epting/3, Schwyz/2, Grüne/2, Patroni/2, Zug/2, Dietikon/2, Kantons Zürich/1, Erbanfällen/1, Erbanfälle/1, Uri/1, Obwalden/1, Nidwalden/1, Wallis/1, Kanton Schaffhausen/1, Liegenschaftenbesitzer/1, Remo Patroni/1, FPS/1, Uster/1, Karl Epting/1, Thomas Büchi/1, Bundes/1, Kanton Graubünden/1, Erbschaftssteuer/1, Aktiengewinne/1, Ulrich Schäpper/1, SP/1, der Schweiz/1, Bruno Kuhn/1, Lindau/1, Herrn Schäpper/1, Staatssteuerprozente Staats-Steuerprozente/1, Kanton Schwyz/1, Regula Pfister/1, FDP/1, Einnahmenausfall/1, Innerschweizer/1, Erbschaftsbereich/1, Bruno Zuppiger/1, Hinwil/1, Schaffhausen/1, Germain Mittaz/1, CVP/1, Pfister/1, Herrn Zuppiger/1, Hans Wiederkehr/1, Büchi/1, Ratsmitgliedern/1


...

[('PER', 24), ('ORG', 18), ('LOC', 11)]
Blum/4, Zürich/3, SP/3, EVP/3, SVP/3, Hans-Jakob Mosimann/2, Winterthur/2, FDP/2, Mosimann/2, Troesch/2, Fehr/2, NZZ/1, Raphael Huber/1, Martin Forster/1, Franziska Troesch-Schnyder/1, Zollikon/1, Peter Reinhard/1, Kloten/1, Esther Holm/1, Grüne/1, Horgen/1, Reinhard/1, Hans Fehr/1, Eglisau/1, Lucius Dürr/1, CVP/1, Alfred Escher-Strasse/1, Hartmuth Attenhofer/1, Hotel International/1, Zürich-Oerlikon/1, Hotels International/1, Dürr/1, Gottlob/1, Blocher/1, Frey/1, Janos Blum/1


## Doc #1

[('PER', 16), ('ORG', 14), ('LOC', 33)]
Zürich/11, SVP/4, Luzern/3, Schwyz/3, Epting/3, Schaffhausen/2, FPS/2, Grüne/2, Patroni/2, Schäpper/2, Zug/2, Dietikon/2, Uri/1, Obwalden/1, Nidwalden/1, Wallis/1, Remo Patroni/1, Uster/1, Karl Epting/1, Thomas Büchi/1, Graubünden/1, Ulrich Schäpper/1, SP/1, SonntagsZeitung/1, Schweiz/1, Bruno Kuhn/1, Lindau/1, Regula Pfister/1, FDP/1, Bruno Zuppiger/1, Hinwil/1, Germain Mittaz/1, CVP/1, Pfister/1, Zuppiger/1, Hans Wiederkehr/1, Büchi/1


## Doc #2

[('PER', 68), ('ORG', 52), ('LOC', 132)]
Pöschwies/34, Zürich/34, Kloten/17, Regensdorf/16, Schweiz/10, FDP/7, SP/6, Holm/5, Bundesrat/4, Grüne/4, SVP/4, Spieler/4, Rudolf Jeker/3, Willy Spieler/3, Mario Fehr/3, Grünen/3, EVP/3, EJPD/2, Küsnacht/2, Sozialdemokratische Fraktion/2, Adliswil/2, Dielsdorf/2, Rietiker/2, Daniel Vischer/2, Noah/2, Aisslinger/2, Leuenberger/2, Kantonsschule Bülach/2, Homberger/2, FraP/2, Mossdorf/2, Vischer/2, Peter Grau/2, SD/2, Neukomm/2, Robert Neukomm/2, Styger/2, ANAG/1, Genf/1, Esther Holm/1, Horgen/1, Tages-Anzeigers/1, Rote Kreuz/1, Germain Mittaz/1, CVP/1, Dietikon/1, Dorothée Fierz/1, Egg/1, Weinland/1, Alt-Pfäffikon/1, Kaspar Günthardt/1, Dällikon/1, Südafrika/1, Robert Rietiker/1, Maur/1, Susanne Frutig/1, Sozialdemokratisch-Gewerkschaftliche Fraktion/1, Pfäffikon/1, Willy/1, Peter Aisslinger/1, Günthardt/1, Fierz/1, Gut/1, Frutig/1, Bern/1, Werner Scherrer/1, Uster/1, Anjuska Weil/1, Zürichs/1, Jugoslawien/1, Sri Lanka/1, Algerien/1, Europa/1, Bä

# Anwendung von Modellen ohne spaCy-Integration

Einschränkungen: Erkannte NE-Spans werden einem spaCy-Dokument zugeordnet, um hier ebenfalls die displaCy-Visualisierung nutzen zu können. Spans, die sich aufgr. Tokenisierungunterschieden nicht zuordnen lassen, werden gesondert ausgegeben. SpacyTextSplitter kann Chunks produzieren, welche die Maximallänge des Modells übersteigen.

## SwissBERT

In [11]:
# Instantiating text splitter from SwissBert tokenizer
text_splitter = SpacyTextSplitter.from_huggingface_tokenizer(
    swissbert.tokenizer,
    separator=" ",
    chunk_size=256,
    chunk_overlap=0,
    pipeline="de_core_news_lg",
)
# Process
for idx, text in enumerate(texts):
    start = time.time()
    display(Markdown("## Doc #{}".format(idx)))

    # Split text into segments
    segments = text_splitter.split_text(text)

    # Add linebreaks for readability
    segments = [segment + "\n\n" for segment in segments]

    seg_docs = []

    # Extract entities per segment
    for segment in segments:
        # Extract entities
        result = swissbert(segment)

        for entity in result:
            entity["label"] = entity.pop("entity_group")

        annotations = [
            entity for entity in result if entity["label"] in {"PER", "ORG", "LOC"}
        ]

        # Create spaCy document
        seg_doc = nlp_blank(segment)
        ents = []

        # Filter NEs and handle mapping of span annotations to spaCy token-level annotations
        for annotation in annotations:
            if annotation["label"] not in {"PER", "LOC", "ORG"}:
                continue
            span = seg_doc.char_span(
                annotation["start"] + 1, annotation["end"], label=annotation["label"]
            )
            if span is None:
                span = seg_doc.char_span(
                    annotation["start"], annotation["end"], label=annotation["label"]
                )
            if span is not None:
                ents.append(span)
            else:
                print("Span error:", annotation)

        seg_doc.ents = spacy.util.filter_spans(ents)
        seg_docs.append(seg_doc)

    # Combine per-segment spaCy documents
    doc = Doc.from_docs(seg_docs)
    result_json_dict["swissbert"].append(get_ent_json(doc))
    stop = time.time()
    duration = stop - start

    # Visualize
    print(
        sorted(
            Counter((ent.label_ for ent in doc.ents)).items(),
            key=lambda x: x[0],
            reverse=True,
        )
    )

    print(
        ", ".join(
            list(
                map(
                    lambda x: "{}/{}".format(x[0], x[1]),
                    sorted(
                        Counter((ent.text for ent in doc.ents)).items(),
                        key=lambda x: x[1],
                        reverse=True,
                    ),
                )
            )
        )
    )
    print()
    print("Time: {:.2f} sec".format(duration))
    displacy.render(doc, style="ent", jupyter=True)

## Doc #0

Span error: {'score': 0.99968004, 'word': 'Hart', 'start': 643, 'end': 648, 'label': 'PER'}
Span error: {'score': 0.9987632, 'word': 'mut', 'start': 648, 'end': 651, 'label': 'PER'}
Span error: {'score': 0.959829, 'word': 'h Attenhofer', 'start': 651, 'end': 663, 'label': 'PER'}
Span error: {'score': 0.72949684, 'word': '.', 'start': 898, 'end': 899, 'label': 'PER'}
[('PER', 19), ('ORG', 11), ('LOC', 13)]
Zürich/3, SP/3, EVP/3, Hans-Jakob Mosimann/2, Winterthur/2, FDP/2, Frau Troesch/2, SVP/2, Blum/1, Raphael Huber/1, Martin Forster/1, Franziska Troesch-Schnyder/1, Zollikon/1, Peter Reinhard/1, Kloten/1, Esther Holm/1, Horgen/1, Mosimann/1, Reinhard/1, Hans Fehr/1, Eglisau/1, Lucius Dürr/1, CVP/1, Alfred Escher-Strasse/1, Hotel International/1, Zürich-Oerlikon/1, Hotels International/1, Herr Fehr/1, Fehr/1, Herr Blocher/1, Herr Frey/1, Janos Blum/1

Time: 3.41 sec


## Doc #1

Span error: {'score': 0.5028198, 'word': 'Patron', 'start': 484, 'end': 491, 'label': 'PER'}
Span error: {'score': 0.99950397, 'word': '', 'start': 690, 'end': 691, 'label': 'PER'}
Span error: {'score': 0.6675605, 'word': 'Zuppig', 'start': 969, 'end': 976, 'label': 'PER'}
[('PER', 9), ('ORG', 7), ('LOC', 33)]
Kanton Zürich/7, SVP/4, Luzern/3, Zürich/3, Schwyz/2, Zug/2, Dietikon/2, Kantons Zürich/1, Uri/1, Obwalden/1, Nidwalden/1, Wallis/1, Kanton Schaffhausen/1, Remo Patroni/1, Uster/1, Karl Epting/1, Thomas Büchi/1, Grüne/1, Kanton Graubünden/1, Ulrich Schäpper/1, SP/1, Schweiz/1, Bruno Kuhn/1, Lindau/1, Kanton Schwyz/1, Regula Pfister/1, FDP/1, Innerschweizer/1, Bruno Zuppiger/1, Hinwil/1, Schaffhausen/1, Germain Mittaz/1, Hans Wiederkehr/1

Time: 1.85 sec


## Doc #2

Span error: {'score': 0.99044925, 'word': '', 'start': 1103, 'end': 1104, 'label': 'ORG'}
Span error: {'score': 0.6491486, 'word': 'Sozialdemokrat', 'start': 529, 'end': 544, 'label': 'ORG'}
Span error: {'score': 0.9996055, 'word': '', 'start': 527, 'end': 528, 'label': 'PER'}
Span error: {'score': 0.6183646, 'word': 'P', 'start': 461, 'end': 463, 'label': 'LOC'}
Span error: {'score': 0.40637845, 'word': 'öschwies', 'start': 463, 'end': 471, 'label': 'LOC'}
Span error: {'score': 0.99964833, 'word': 'Do', 'start': 964, 'end': 967, 'label': 'PER'}
Span error: {'score': 0.9986627, 'word': 'roth', 'start': 967, 'end': 971, 'label': 'PER'}
Span error: {'score': 0.9965439, 'word': 'ée Fierz', 'start': 971, 'end': 979, 'label': 'PER'}
Span error: {'score': 0.6065748, 'word': 'Hol', 'start': 760, 'end': 764, 'label': 'PER'}
Span error: {'score': 0.6500239, 'word': 'Kaserne', 'start': 417, 'end': 425, 'label': 'LOC'}
Span error: {'score': 0.48783496, 'word': 'Riet', 'start': 628, 'end': 633, 'l

## Flair fl_german_large

In [15]:
# Instantiate text splitter using spaCy model
text_splitter = SpacyTextSplitter(
    separator=" ", chunk_size=256, chunk_overlap=0, pipeline="de_core_news_lg"
)

# Process
for idx, text in enumerate(texts):
    start = time.time()
    display(Markdown("## Doc #{}".format(idx)))

    # Split text into segments
    segments = text_splitter.split_text(text)

    # Add linebreaks for readability
    segments = [segment + "\n\n" for segment in segments]

    seg_docs = []

    # Extract entities per segment
    for segment in segments:
        result = Sentence(segment)
        fl_german_large.predict(result)

        # Map flairNLP entity to spaCy-compatible span dictionary
        annotations = [
            {
                "span": entity.text,
                "label": entity.tag,
                "score": entity.score,
                "start": entity.start_position,
                "end": entity.end_position,
            }
            for entity in result.get_spans("ner")
            if entity.tag in {"PER", "ORG", "LOC"}
        ]
        # Create spaCy document
        seg_doc = nlp_blank(segment)
        ents = []

        # Filter NEs and handle mapping of span annotations to spaCy token-level annotations
        for annotation in annotations:
            if annotation["label"] not in {"PER", "LOC", "ORG"}:
                continue
            span = seg_doc.char_span(
                annotation["start"] + 1, annotation["end"], label=annotation["label"]
            )
            if span is None:
                span = seg_doc.char_span(
                    annotation["start"], annotation["end"], label=annotation["label"]
                )
            if span is not None:
                ents.append(span)
            else:
                print("Span error:", annotation)

        seg_doc.ents = spacy.util.filter_spans(ents)
        seg_docs.append(seg_doc)

    # Combine per-segment spaCy documents
    doc = Doc.from_docs(seg_docs)
    result_json_dict["fl_german_large"].append(get_ent_json(doc))
    stop = time.time()
    duration = stop - start

    # Visualize
    print(
        sorted(
            Counter((ent.label_ for ent in doc.ents)).items(),
            key=lambda x: x[0],
            reverse=True,
        )
    )

    print(
        ", ".join(
            list(
                map(
                    lambda x: "{}/{}".format(x[0], x[1]),
                    sorted(
                        Counter((ent.text for ent in doc.ents)).items(),
                        key=lambda x: x[1],
                        reverse=True,
                    ),
                )
            )
        )
    )
    print()
    print("Time: {:.2f} sec".format(duration))
    displacy.render(doc, style="ent", jupyter=True)

## Doc #0

Created a chunk of size 271, which is longer than the specified 256
Created a chunk of size 291, which is longer than the specified 256
Created a chunk of size 290, which is longer than the specified 256


[('PER', 24), ('ORG', 15), ('LOC', 13)]
Blum/4, Zürich/3, SP/3, EVP/3, SVP/3, Hans-Jakob Mosimann/2, Winterthur/2, FDP/2, Mosimann/2, Troesch/2, Fehr/2, Schweiz/1, Raphael Huber/1, Martin Forster/1, Franziska Troesch-Schnyder/1, Zollikon/1, Peter Reinhard/1, Kloten/1, Esther Holm/1, Grüne/1, Horgen/1, Reinhard/1, Hans Fehr/1, Eglisau/1, Lucius Dürr/1, CVP/1, Alfred Escher-Strasse/1, Hartmuth Attenhofer/1, Hotel International/1, Zürich-Oerlikon/1, Hotels International/1, Dürr/1, Blocher/1, Frey/1, Janos Blum/1

Time: 13.90 sec


## Doc #1

Created a chunk of size 257, which is longer than the specified 256
Created a chunk of size 354, which is longer than the specified 256


[('PER', 17), ('ORG', 13), ('LOC', 31)]
Zürich/11, SVP/4, Luzern/3, Schwyz/3, Epting/3, Schaffhausen/2, Grüne/2, Patroni/2, Schäpper/2, Zug/2, Dietikon/2, Uri/1, Obwalden/1, Nidwalden/1, Wallis/1, Remo Patroni/1, FPS/1, Karl Epting/1, Thomas Büchi/1, Graubünden/1, Ulrich Schäpper/1, SP/1, SonntagsZeitung/1, Schweiz/1, Bruno Kuhn/1, Lindau/1, Regula Pfister/1, FDP/1, Bruno Zuppiger/1, Hinwil/1, Germain Mittaz/1, CVP/1, Pfister/1, Zuppiger/1, Hans Wiederkehr/1, Büchi/1

Time: 5.48 sec


## Doc #2

Created a chunk of size 301, which is longer than the specified 256
Created a chunk of size 273, which is longer than the specified 256
Created a chunk of size 353, which is longer than the specified 256
Created a chunk of size 342, which is longer than the specified 256
Created a chunk of size 277, which is longer than the specified 256
Created a chunk of size 289, which is longer than the specified 256
Created a chunk of size 259, which is longer than the specified 256
Created a chunk of size 341, which is longer than the specified 256
Created a chunk of size 270, which is longer than the specified 256
Created a chunk of size 283, which is longer than the specified 256
Created a chunk of size 274, which is longer than the specified 256
Created a chunk of size 260, which is longer than the specified 256
Created a chunk of size 293, which is longer than the specified 256
Created a chunk of size 289, which is longer than the specified 256
Created a chunk of size 289, which is longer tha

[('PER', 70), ('ORG', 43), ('LOC', 130)]
Pöschwies/35, Zürich/34, Regensdorf/16, Kloten/12, Schweiz/10, FDP/7, SP/6, Holm/5, Spieler/5, ANAG/4, Grüne/4, Willy Spieler/4, SVP/4, Rudolf Jeker/3, Mario Fehr/3, Dielsdorf/3, Grünen/3, Bülach/3, EVP/3, EJPD/2, Küsnacht/2, Adliswil/2, Rietiker/2, Frutig/2, Daniel Vischer/2, Gut/2, Aisslinger/2, Leuenberger/2, Homberger/2, Waid/2, Weil/2, FraP!/2, Mossdorf/2, Vischer/2, Peter Grau/2, SD/2, Neukomm/2, Robert Neukomm/2, Styger/2, Genf/1, Esther Holm/1, Horgen/1, Bundesrat/1, Rote Kreuz/1, Germain Mittaz/1, CVP/1, Dietikon/1, Dorothée Fierz/1, Egg/1, Kaspar Günthardt/1, Dällikon/1, Südafrika/1, Robert Rietiker/1, Maur/1, Susanne Frutig/1, Peter Aisslinger/1, Günthardt/1, Fierz/1, Arche Noah/1, Noah/1, Bern/1, Werner Scherrer/1, Uster/1, Anjuska/1, Zürichs/1, Jugoslawien/1, Sri Lanka/1, Algerien/1, Europa/1, Martin Mossdorf/1, Laurenz Styger/1, Erhard Bernet/1, FPS/1, Freiburg/1, Moritz Leuenberger/1, Fehr/1, Ernst Homberger/1, Bund/1

Time: 20.90

# Export der Annotation als JSON zwecks Diff (separates Notebook)

In [16]:
datestr = datetime.now().replace(microsecond=0).isoformat()

with open("qualitative_evaluation_0_results_{}.json".format(datestr.replace(':','_')), "w") as f:
    f.write(json.dumps(result_json_dict, indent=4, ensure_ascii=False))

# Resultate

## Grobübersicht

* **spaCy de_core_news_lg** erzeugt viele grob falsche False Positives (normale Nomen, Phrasen mit Adjektiven oder Adverben, ungewöhnliche oder falsch geschriebene Nichtnomen, auch mit kleinem Anfangsbuchstaben)
* **SwissBert** weist gute Präzision auf Spans aus, die mit der Spacy-Tokenisierung (für spacy.blank("de")) zusammenfallen, erzeugt jedoch Artefakte auf Subtokenebene (trotz Aggregation der Subtoken im Output), inklusive Annotation von einzelnen Buchstaben und sogar leeren Strings. SwissBert bietet im Gegensatz zu SpanMarker keine direkte spaCy-Integration.
* **Flair/ner-german-large**
  * wirkt informell ähnlich präzise wie SwissBert, bietet jedoch den Vorteil einer Framework-Lösung (Textchunking, etc.)
  * verursacht keine Span Errors; Tokenisierung weitgehend kompatibel mit spaCy
  * Schnellste spaCy-Alternative
* **SpanMarker** ist, je nach Modell, gleich schnell wie Swissbert bis dreimal langsamer, bei in etwa ähnlicher Leistung und schlechterer Skalierung mit längeren Dokumenten. Die Unterschiede zwischen den vier SpanMarker-Modellen sind geringfügig, wobei Gelectra am besten abschneidet. **SpanMarker kann direkt in spaCy eingebunden werden**: https://github.com/tomaarsen/SpanMarkerNER#using-pretrained-spanmarker-models-with-spacy

## Informeller Diff der Resultate - erster Eindruck

 * SpanMarker Gelectra ist den anderen SpanMarker-Modellen überlegen, allerdings auch doppelt so rechenintensiv und damit ingesamt das mit Abstand teuerste Modell
 * de_core_news_lg weist gegenüber Gelectra relativ wenige FN, dafür zahlreiche FP auf.
 * SwissBERT hat hohe Präzision, jedoch geringe Ausbeute (viele FN bei PER) 
 * Flair schneidet leicht schlechter als SpanMarker Gelectra ab, bei deutlich besserer Laufzeit

## Laufzeiten (in Sekunden)

Anmerkungen: Bei den Laufzeiten der HF-Modelle handelt es sich um Angaben ohne spaCy-Einbindung, die ungeachtet der Modellgrösse einen Overhead von 33-50% verursacht, mit der sich allerdings auch das Chunking erübrigt (Satztokenisierung).

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-uca5{text-align:left;text-decoration:underline;vertical-align:top}
.tg .tg-dvid{background-color:#efefef;border-color:inherit;font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-fymr{border-color:inherit;font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-b3sw{background-color:#efefef;font-weight:bold;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky"></th>
    <th class="tg-dvid">Doc1</th>
    <th class="tg-dvid">Doc2</th>
    <th class="tg-dvid">Doc3</th>
</thead>
<tbody>
    <tr>
    <td class="tg-dvid"><i>Token</i></td>
    <td class="tg-0pky"><i>2'079</i></td>
    <td class="tg-0pky"><i>1'757</i></td>
    <td class="tg-0pky"><i>10'508</i></td>
  </tr>
  <tr>
    <td class="tg-dvid">de_core_news_md</td>
    <td class="tg-0pky">0.07</td>
    <td class="tg-0pky">0.05</td>
    <td class="tg-0pky">0.29</td>
  </tr>
  <tr>
    <td class="tg-dvid">de_core_news_lg</td>
    <td class="tg-fymr">0.07</td>
    <td class="tg-fymr">0.05</td>
    <td class="tg-fymr">0.29</td>
  </tr>
  <tr><td/><td/><td/><td/></tr>
  <tr>
    <td class="tg-dvid">swissbert</td>
    <td class="tg-0pky">2.37</td>
    <td class="tg-0pky">1.53</td>
    <td class="tg-0pky">9.14</td>
  </tr>
  <tr>
    <td class="tg-dvid">sm_gelectra</td>
    <td class="tg-0pky">7.18</td>
    <td class="tg-0pky">5.22</td>
    <td class="tg-0pky">30.36</td>
  </tr>
  <tr>
    <td class="tg-dvid">sm_teams</td>
    <td class="tg-0pky">4.13</td>
    <td class="tg-0pky">2.69</td>
    <td class="tg-0pky">15.60</td>
  </tr>
  <tr>
    <td class="tg-dvid">sm_bert</td>
    <td class="tg-0pky">2.96</td>
    <td class="tg-0pky">2.40</td>
    <td class="tg-0pky">14.49</td>
  </tr>
  <tr>
    <td class="tg-dvid">sm_td_bert</td>
    <td class="tg-0pky">2.90</td>
    <td class="tg-0pky">2.46</td>
    <td class="tg-0pky">14.61</td>
  </tr>
  <tr>
    <td class="tg-dvid">fl_german_large<br>(w/ external chunking)</td>
    <td class="tg-0pky">9.72</td>
    <td class="tg-0pky">4.79</td>
    <td class="tg-0pky">22.20</td>
  </tr>
  <tr>
    <td class="tg-dvid">fl_german_large<br>(w/o external chunking)<br></td>
    <td class="tg-0pky"><b>2.09</b></td>
    <td class="tg-0pky"><b>0.67</b></td>
      <td class="tg-0pky"><b>5.47</b></b></td>
  </tr>
</tbody>
</table>

# Entscheidungen hinsichtlich quantitativer Evaluation

* Aus Gründen der Praktikabilität erfolgt eine Einschränkung auf folgende **drei Modelle** für die quantitative Erstevaluation.
    1. SpanMarker Gelectra
    2. FlairNLP
    3. spaCy de_core_news_lg (als Baseline, Potenzial Fine-Tuning)